In [26]:
def calculateDims(coordinates:list): 
    #Remember that the coordinates are written like x1;y1;x2;y2;x3;y3
    print(coordinates) #Remove after 
    if type(coordinates)!=list:
        print(type(coordinates))
        raise ValueError
    centerWidthHeights = [] #The format will be [[center0,width0,height0],[center1,width1,height1],.........]
    cenwidhgt = [] #These are the [centerX,widthX,heightX] described above
                   #centerX will be written as xX,yX => [centerX,widthX,heightX]  = [xX,yX,widthX,heightX]
    xs = []
    ys = []
    
    coords = coordinates[0]
    coordslist = coords.split(";") #Remember that every value indexed at a even num is xs, and odd num is ys
    for i in range(len(coordslist)):
        if i%2==0: 
            xs.append(int(coordslist[i]))
        else:
            ys.append(int(coordslist[i]))
    xsMax = max(xs)
    xsMin = min(xs)
    ysMax = max(ys)
    ysMin = min(ys)     
    xsCen = (xsMin+xsMax)/2
    ysCen = (ysMin+ysMax)/2
    height = abs(ysMax-ysMin) #should be positive, but just to be sure
    width = abs(xsMax-xsMin)
    cenwidhgt = [xsCen,ysCen,width,height]
    centerWidthHeights.append(cenwidhgt)
    xs = []
    ys = []
    print(centerWidthHeights) #Remove aferwars
    return centerWidthHeights

In [27]:
#Read in the textfile with wher the nodule imformation is pou up like this: 
    # NoduleID: SOPInstanceUID: png path: coordinates: 
    # Ex:
    # Nodule 01,1.3.2.4.6.3.12413......,/home/heris/............../image1.png,x1;y1;x2;y2;x3;y3

def boundingBoxDim(input_txt_path:str, output_txt_path:str):
    try:
        NoduleIDs = []
        SOPInstanceUIDs = []
        img_paths = []
        CoordinatesTot = []
        with open(input_txt_path, "r") as txt:
            lines = txt.readlines()
            for line in lines:
                #Each line is in a csv format so can split on ","
                line_split = line.split(",")
                noduleID = line_split[0]
                NoduleIDs.append(noduleID)   
                sopInstanceUID = line_split[1]
                SOPInstanceUIDs.append(sopInstanceUID)
                img_path = line_split[2]
                img_paths.append(img_path)
                coordinates = line_split[3]
                CoordinatesTot.append(coordinates) 
        centersWidthsHeigths = calculateDims(CoordinatesTot)
    except FileNotFoundError:
        print(f"File {input_txt_path} not found.")
    except SyntaxError:
        print(f"Wrong imput syntax!")
    
    #JASON or TXT, read up on YOLO v8
    try:
        with open(output_txt_path, 'w') as file:
            # Write the content to the file
            
            #Lesion id = 1
            for dims in centersWidthsHeigths:
                file.write("1 ") #Lesion ID = 1
                for dim in dims: 
                    if dim != dims[-1]:
                        file.write(f"{dim} ")
                    else: 
                        file.write(f"{dim}\n")
            
        print(f"Text has been written to {output_txt_path}.")
    except Exception as e:
        print(f"An error occurred: {e}")

    return NoduleIDs, SOPInstanceUIDs, img_paths #return another element, something with coordinateinformation 

In [28]:
boundingBoxDim("/home/heris/data/RAW/LIDC-IDRI-dicom/LIDC-IDRI-0001/image_1.3.6.1.4.1.14519.5.2.1.6279.6001.110383487652933113465768208719.txt","/home/heris/data/RAW/test1.txt")

['312;355;311;356;310;357;309;357;308;358;308;359;308;360;307;360;306;361;306;362;305;363;304;364;303;365;303;366;302;367;302;368;302;369;301;370;301;371;300;371;299;372;299;373;299;374;299;375;299;376;300;377;301;378;302;379;303;379;304;379;305;379;306;379;307;378;308;377;308;376;309;375;310;375;311;375;312;375;313;375;314;375;315;375;316;375;317;375;318;375;319;375;320;374;321;373;322;372;322;371;322;370;323;369;324;368;325;367;326;366;327;365;328;364;328;363;327;362;327;361;326;360;325;359;324;359;323;358;322;358;321;357;320;358;319;358;318;358;318;357;317;356;316;355;315;355;314;355;313;355;312;355\n', '318;374;319;373;320;372;321;371;322;372;323;371;324;371;325;370;325;369;325;368;326;367;326;366;326;365;325;364;324;364;323;364;322;364;321;363;320;362;319;362;318;361;317;362;316;362;315;361;315;360;314;359;313;359;312;360;312;361;312;362;311;363;310;363;309;364;308;365;308;366;307;367;307;368;307;369;308;370;309;370;310;369;311;369;312;370;313;371;314;371;315;370;316;371;316;372;3

(['Nodule 001', 'MI014_12127'],
 ['1.3.6.1.4.1.14519.5.2.1.6279.6001.110383487652933113465768208719',
  '1.3.6.1.4.1.14519.5.2.1.6279.6001.110383487652933113465768208719'],
 ['/home/heris/data/RAW/LIDC-IDRI-dicom/image_1.3.6.1.4.1.14519.5.2.1.6279.6001.110383487652933113465768208719.png',
  '/home/heris/data/RAW/LIDC-IDRI-dicom/image_1.3.6.1.4.1.14519.5.2.1.6279.6001.110383487652933113465768208719.png'])

In [29]:
boundingBoxDim("/home/heris/data/RAW/LIDC-IDRI-dicom/LIDC-IDRI-0001/image_1.3.6.1.4.1.14519.5.2.1.6279.6001.202709423777326615340853838834.txt","/home/heris/data/RAW/test2.txt")

['310;351;309;352;309;353;309;354;308;354;307;354;306;355;305;355;304;354;303;355;303;356;303;357;303;358;303;359;303;360;302;361;301;362;301;363;301;364;301;365;301;366;301;367;301;368;302;369;303;369;304;370;304;371;303;372;302;372;301;373;301;374;302;375;303;375;304;375;305;376;306;377;307;377;307;378;307;379;308;380;309;381;310;382;311;381;312;380;313;380;314;379;315;378;316;378;317;378;318;378;319;378;320;379;321;379;322;379;323;378;323;377;324;376;325;376;326;376;327;377;327;378;328;379;329;379;330;379;331;379;332;378;332;377;332;376;332;375;332;374;332;373;333;372;334;371;334;370;334;369;334;368;333;367;333;366;332;365;331;364;331;363;330;362;330;361;330;360;330;359;330;358;330;357;330;356;330;355;330;354;330;353;329;352;328;353;327;354;326;354;325;354;324;354;323;354;322;354;321;354;320;354;319;354;319;353;319;352;318;351;317;352;317;353;317;354;316;354;315;354;314;355;313;355;313;354;313;353;313;352;312;351;311;351;310;351\n', '329;379;330;378;331;378;332;377;333;376;333;375;3

(['Nodule 001', 'IL057_127364', 'MI014_12127', '0'],
 ['1.3.6.1.4.1.14519.5.2.1.6279.6001.202709423777326615340853838834',
  '1.3.6.1.4.1.14519.5.2.1.6279.6001.202709423777326615340853838834',
  '1.3.6.1.4.1.14519.5.2.1.6279.6001.202709423777326615340853838834',
  '1.3.6.1.4.1.14519.5.2.1.6279.6001.202709423777326615340853838834'],
 ['/home/heris/data/RAW/LIDC-IDRI-dicom/image_1.3.6.1.4.1.14519.5.2.1.6279.6001.202709423777326615340853838834.png',
  '/home/heris/data/RAW/LIDC-IDRI-dicom/image_1.3.6.1.4.1.14519.5.2.1.6279.6001.202709423777326615340853838834.png',
  '/home/heris/data/RAW/LIDC-IDRI-dicom/image_1.3.6.1.4.1.14519.5.2.1.6279.6001.202709423777326615340853838834.png',
  '/home/heris/data/RAW/LIDC-IDRI-dicom/image_1.3.6.1.4.1.14519.5.2.1.6279.6001.202709423777326615340853838834.png'])